2D visualisation fo the analytic parameter and behavior space. (Fig. 22, 23 of supplementary material)

In [ ]:
# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier

height_default_1 = int(3.5 * pixel_cm_ration) * multiplier
height_default_2 = int(4.5 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 0 * multiplier 
left_margin = 0 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 0 * multiplier 

font_size = 10 * multiplier 
font_family='Times New Roman'

line_width = 2 * multiplier 

In [ ]:
# Define and load data
import autodisc as ad
import ipywidgets
import plotly
import numpy as np
import collections
plotly.offline.init_notebook_mode(connected=True)

data_filters = collections.OrderedDict()
data_filters['dead'] = ('classifier_dead.data', '==', True)
data_filters['non-animal'] = (('classifier_dead.data', '==', False), 'and', ('classifier_animal.data', '==', False))
data_filters['animal'] = ('classifier_animal.data', '==', True)

org_experiment_definitions = dict()

org_experiment_definitions['main_paper'] = [
     
    dict(id = '1',
         directory = '../experiments/experiment_000001',
         name = 'Random',
         is_default = True),

    dict(id = '109',
         directory = '../experiments/experiment_000109',
         name = 'IMGEP-HGS',
         is_default = True),
    
    dict(id = '202',
         directory = '../experiments/experiment_000202',
         name = 'IMGEP-PGL',
         is_default = True),

    dict(id = '302',
         directory = '../experiments/experiment_000302',
         name = 'IMGEP-OGL',
         is_default = True),
]

repetition_ids = list(range(1))

# define names and load the data
experiment_name_format = '<name>' # <id>, <name>

#global experiment_definitions
experiment_definitions = []
experiment_statistics = []

current_experiment_list = 'main_paper'

experiment_definitions = []
for org_exp_def in org_experiment_definitions[current_experiment_list]:
    new_exp_def = dict()
    new_exp_def['directory'] = org_exp_def['directory']
    if 'is_default' in org_exp_def:
        new_exp_def['is_default'] = org_exp_def['is_default']

    if 'name' in org_exp_def:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id'], 'name': org_exp_def['name']})
    else:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id']})

    experiment_definitions.append(new_exp_def)

experiment_statistics = dict()
for experiment_definition in experiment_definitions:
    experiment_statistics[experiment_definition['id']] = ad.gui.jupyter.misc.load_statistics(experiment_definition['directory'])
       

In [ ]:
# Parameters
run_parameter_space_dimensions = [
    ('run_parameters', 'T'),
    ('run_parameters', 'R'),
    ('run_parameters', 'm'),
    ('run_parameters', 's'),
    ('run_parameters', 'b', 0),
    ('run_parameters', 'b', 1),
    ('run_parameters', 'b', 2),
    ('parameter_initstate_space_representation','data','[0]'),
    ('parameter_initstate_space_representation','data','[1]'),
    ('parameter_initstate_space_representation','data','[2]'),
    ('parameter_initstate_space_representation','data','[3]'),
    ('parameter_initstate_space_representation','data','[4]'),
    ('parameter_initstate_space_representation','data','[5]'),
    ('parameter_initstate_space_representation','data','[6]'),
    ('parameter_initstate_space_representation','data','[7]'),
]

statistic_space_dimensions = [
    ('lenia_statistics','statistics.activation_mass[-1]'),
    ('lenia_statistics','statistics.activation_volume[-1]'),
    ('lenia_statistics','statistics.activation_density[-1]'),
    ('lenia_statistics','statistics.activation_mass_asymmetry[-1]'),
    ('lenia_statistics','statistics.activation_mass_distribution[-1]'),
    ('statistic_space_representation','data','[0]'),
    ('statistic_space_representation','data','[1]'),
    ('statistic_space_representation','data','[2]'),
    ('statistic_space_representation','data','[3]'),
    ('statistic_space_representation','data','[4]'),
    ('statistic_space_representation','data','[5]'),
    ('statistic_space_representation','data','[6]'),
    ('statistic_space_representation','data','[7]'),
]

In [ ]:
## Compute PCA+UMAP per experiment repetition
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing
import collections
import sklearn

pca_algorithm = PCA(n_components=2)


def get_statistic_space_points_of_repetition(experiment_definitions, source_data, space_defintion, data_filter=None):
    '''
    Collects for each exploration run the point in a defined space and constructs a matrix holding all points.
    Collects this data for each experiment.
    '''
    
    data_filter_inds = None
    if data_filter is not None and data_filter:
        # filter data according data_filter the given filter
        data_filter_inds = ad.gui.jupyter.misc.filter_experiments_data(source_data, data_filter)
    
    data_statistic_space_points = dict() # {experiment-id} [repetition_idx, run_idx, space_dim_idx]
    
    for exp_def in experiment_definitions:
        exp_id = exp_def['id']
        
        rep_data_matricies = []
        
        cur_data_filter_inds = data_filter_inds[exp_id] if data_filter_inds is not None else None  

        # load data and define the bin_config
        for dim_name in space_defintion:

            # get all repetition data for the current parameter
            try:
                cur_data = ad.gui.jupyter.misc.get_experiment_data(data=source_data, experiment_id=exp_id, data_source=dim_name, repetition_ids='all', data_filter_inds=cur_data_filter_inds)
                
            except Exception as err:
                if not isinstance(err, KeyError):
                    raise Exception('Error during loading of data for Experiment {!r} (Datasource = {!r} )!'.format(exp_id, dim_name)) from err
                else:
                    # could not load data
                    warnings.warn('Could not load data for Experiment {!r} (Datasource = {!r} )!'.format(exp_id, dim_name))
                    cur_data = []
            
            # go over repetitions
            for rep_idx, cur_rep_data in enumerate(cur_data):
                cur_rep_data = np.array([cur_rep_data]).transpose()

                if rep_idx >= len(rep_data_matricies):
                    rep_data_matricies.append(cur_rep_data)
                else:
                    rep_data_matricies[rep_idx] = np.hstack((rep_data_matricies[rep_idx], cur_rep_data))
        
        data_statistic_space_points[exp_id] = dict()
        data_statistic_space_points[exp_id]['statistic_space_points'] = np.array(rep_data_matricies)

    return data_statistic_space_points


def do_dimension_reduction(algorithm, space_dimensions, data_filter=None):
    
    # data_statistic_space_points:  {experiment-id}{'statistic_space_points'}[repetition_idx, run_idx, space_dim_idx]
    data_statistic_space_points = get_statistic_space_points_of_repetition(experiment_definitions,
                                                                           experiment_statistics,
                                                                           space_dimensions,
                                                                           data_filter=data_filter)
    
    # fit the umap models for each repetition on all the data of all experiments
    umap_data = collections.defaultdict(lambda: dict()) # {experiment_id} {repetition_idx} ndarray[run_idx, dim_idx] 
        
    for repetition_id in repetition_ids:

        # collect the data of all experiments
        all_experiment_data = None # ndarray[run_idx, dim_idx]

        # (start_idx, end_idx) of the data in collected data the correspond to each experiment
        data_idx_per_experiment = dict() # {experimend_id} (start_idx, end_idx)
        
        for exp_id, exp_data in data_statistic_space_points.items():

            if all_experiment_data is None:
                all_experiment_data = exp_data['statistic_space_points'][repetition_id,:,:]
                start_idx = 0
            else:
                start_idx = all_experiment_data.shape[0]
                all_experiment_data = np.vstack((all_experiment_data, exp_data['statistic_space_points'][repetition_id,:,:]))
                
            end_idx = all_experiment_data.shape[0] - 1
            
            data_idx_per_experiment[exp_id] = (start_idx, end_idx)

        
        all_experiment_data = sklearn.preprocessing.MinMaxScaler().fit_transform(all_experiment_data)
        
        all_experiment_data = np.nan_to_num(all_experiment_data)
            
        # fitt all data
        embedded_data = algorithm.fit_transform(np.nan_to_num(all_experiment_data))
        embedded_data = sklearn.preprocessing.MinMaxScaler().fit_transform(embedded_data)
        
        for exp_id in data_statistic_space_points.keys():
            
            start_idx = data_idx_per_experiment[exp_id][0]
            end_idx = data_idx_per_experiment[exp_id][1]
            
            umap_data[exp_id][repetition_id] = embedded_data[start_idx:end_idx+1]
                        
    return umap_data

In [ ]:
# plot functions
import plotly
plotly.offline.init_notebook_mode(connected=True)
    
def plot_single_experiment(data, experiment_id, repetition_idx=0, experiment_statistics=None, data_filters=None, default_color='black', showlegend=True):  
        
    marker_color = ['rgb(0,0,0)', 'rgb(0,158,115)','rgb(230,159,0)', 'rgb(204,121,167)', 'rgb(0,114,178)', 'rgb(230,159,0)']
    marker_symbol = ['circle', 'square', 'diamond']    
    
    plot_data = []
    
    if data_filters is None:
        trace = plotly.graph_objs.Scatter(
            x = data[experiment_id][repetition_idx][0:, 0],
            y = data[experiment_id][repetition_idx][0:, 1],
            mode = 'markers',
            marker = dict(
                color = default_color
            )
        )
        plot_data.append(trace)
        
    else:
        
        # go through filters
        for filter_idx, (data_filter_name, data_filter) in enumerate(data_filters.items()):
            
            filter_inds = ad.gui.jupyter.misc.filter_single_experiment_data(experiment_statistics[experiment_id], data_filter, repetition_idx)
            
            trace = plotly.graph_objs.Scatter(
                x = data[experiment_id][repetition_idx][filter_inds, 0],
                y = data[experiment_id][repetition_idx][filter_inds, 1],
                mode = 'markers',
                name = data_filter_name,
                marker = dict(
                    color = marker_color[filter_idx],
                    symbol = marker_symbol[filter_idx],
                    size = 4,
                )
            )
            plot_data.append(trace)
            
    # general layout of figure
    layout = plotly.graph_objs.Layout(
        xaxis=dict(
            range=[-0.03,1.03],
            showgrid=True,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        yaxis=dict(
            range=[-0.03,1.03],
            showgrid=True,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False
        ),
        font = dict(
            family=font_family, 
            size=font_size, 
            ),
        width=width_half, # in cm
        height=height_default_2, # in cm
        
        margin = dict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),        
        
        showlegend=showlegend,
        legend=dict(
            xanchor='left',
            yanchor='bottom',
            y=0,
            x=0,
            ), 
    )

    fig = plotly.graph_objs.Figure(data=plot_data, layout=layout)

    plotly.offline.iplot(fig)
    
    return fig

# PCA

## Parameter Space

In [ ]:
# get data
parameter_space_pca_data = do_dimension_reduction(pca_algorithm, run_parameter_space_dimensions)

In [ ]:
print('Random')
fig = plot_single_experiment(parameter_space_pca_data, 
                            experiment_id='Random',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)
print('IMGEP-HGS')
fig = plot_single_experiment(parameter_space_pca_data, 
                            experiment_id='IMGEP-HGS',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)

print('IMGEP-PGL')
fig = plot_single_experiment(parameter_space_pca_data, 
                            experiment_id='IMGEP-PGL',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=True)
print('IMGEP-OGL')
fig = plot_single_experiment(parameter_space_pca_data, 
                            experiment_id='IMGEP-OGL',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)

## Statistic Space -All

In [ ]:
# get data
statistic_space_all_pca_data = do_dimension_reduction(pca_algorithm, statistic_space_dimensions)


In [ ]:
print('Random')
fig = plot_single_experiment(statistic_space_all_pca_data, 
                            experiment_id='Random',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)

print('IMGEP-HGS')
fig = plot_single_experiment(statistic_space_all_pca_data, 
                            experiment_id='IMGEP-HGS',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)


print('IMGEP-PGL')
fig = plot_single_experiment(statistic_space_all_pca_data, 
                            experiment_id='IMGEP-PGL',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=True)

print('IMGEP-OGL')
fig = plot_single_experiment(statistic_space_all_pca_data, 
                            experiment_id='IMGEP-OGL',
                            experiment_statistics=experiment_statistics, 
                            data_filters=data_filters,
                            showlegend=False)